# Single step predictor

This notebook shows how to load the predictor, make some predictions, and postprocess them.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fmlwright.modeling.predictor import SingleStepPredictor
from fmlwright.core import preprocessing, postprocessing

from pathlib import Path
import random
import logging

from shapely.geometry import GeometryCollection
import numpy as np
import geopandas as gpd


import cv2
import matplotlib.pyplot as plt

In [ ]:
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(module)s - %(message)s"
)

In [ ]:
categories = ['living_room']

generator_model = Path('../models/complete_floorplan/')
config_file = Path('../config/config_nn_complete_floorplan.yaml')
input_images = Path('../data/images/complete_floorplan/input/')

In [ ]:
Predictor = SingleStepPredictor(model_location=generator_model, 
                                conf_location=config_file, 
                                categories=categories)
Predictor.load_model()

In [ ]:
input_images_list = [x for x in input_images.glob('*.png')]
len(input_images_list)

In [ ]:
random_file = random.choice(input_images_list)

input_image = cv2.imread(str(random_file))
input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)

predictions = Predictor.predict(input_image, 5)
Predictor.visualize_predictions(input_image, predictions=predictions);

In [ ]:
postprocessed_predictions = Predictor.postprocess_predictions(predictions=predictions, input_img=input_image)

In [ ]:
preprocessed_img = preprocessing.resize_and_pad(
    input_image, size=(256, 256), pad_color=255
)

in_img_gdf = preprocessing.image_to_geodataframe(preprocessed_img)
in_img_gdf.geometry = [in_img_gdf.unary_union] * in_img_gdf.shape[0]
in_img_gdf = in_img_gdf.head(1).explode().reset_index(drop=True)

In [ ]:
n_predictions = len(postprocessed_predictions.keys())
fig, axes = plt.subplots(nrows=int(np.ceil(n_predictions/2)), ncols=2, figsize=(30, 30))
axes = axes.flatten()
for _preds, ax in zip(postprocessed_predictions.items(), axes):
    in_img_gdf.exterior.buffer(1).plot(ax=ax, color='black')

    _key, _res = _preds
    _res.plot(color=_res['colors'], ax=ax)
    _res.exterior.plot(color='black', ax=ax)
    ax.set_title(_key)

for ax in axes:
    ax.axis('off')
fig.suptitle('Postprocessed floorplans', fontsize=16, y=1)
plt.tight_layout();